In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

def run_simulations(tau, num_trials):
    
    
    def estBetaParams(mu, var):
        alpha=((1 - mu) / var - 1 / mu) * mu**2
        beta=alpha * (1 / mu - 1)
        return alpha,beta

    def estGammaParams(mu,var):
        shape=(mu**2)/var
        scale=var/mu
        return shape, scale
    
        #Non-Stochastic Parameters Parameters

    N = 10**6           #250000               # Total population
    p_3months = 0.3          # Proportion tested every 3 months
    p_12months = 1-p_3months         # Proportion tested every 12 months
   

    # The (weighted) average testing proportion for our population (proportion per day)
    average_daily_testing_proportion = (p_3months/91 + p_12months/365)
    tests_per_day=N*average_daily_testing_proportion
    # I think that the generic person will get tested with this probability each day, and so the waiting
    # time for a test from any given instant is 1/average_testing_proportion (exponential waiting time)
    # No.... maybe it should be with rate tests_per_day??

    # Initial conditions
    E0 = 0.10 * N             # Initial exposed individuals.
    X0 = N - E0              # Initial susceptible individuals.
    S0 = 0                   # Initial symptomatic individuals (we assume the symptoms are NOTICEABLE). Symptomatic Individuals are assumed to isolate.
    A_u0 = 0               # Initial asymptomatic individuals who are untested.
    A_t0 = 0                 # Initial asympt. who are tested and awating a positive. These individuals will NOT isolate. 
    A_pos0 = 0                 # Initial asympt. positive test. These people are isolating. 
                       # Initial people with complications 
        
        
    #Define the Model 

    def model(Y, t, beta, ppd, epsilon, gamma_t, gamma_u, lambda_, theta, omega_r, average_daily_testing_proportion , tau, N):


        X, E, S, A_u, A_t, A_pos= Y


        # Here, we assume that the only groups driving the spread of infection are the asymptomatic who have not received a positive test
        dXdt = -beta*ppd*X*(A_u+A_t)/N + gamma_t*S  + gamma_t*A_pos + gamma_u*A_t + (1-theta)*gamma_u*A_u


        dEdt = beta*ppd*X*(A_u+A_t)/N - epsilon*E


        dSdt = (1-lambda_)*epsilon*E + theta*gamma_u*A_u - gamma_t*S 


        dA_udt = lambda_*epsilon*E - average_daily_testing_proportion*A_u - gamma_u*A_u #I want some proportion to clear, and some proportion to become infectious

        dA_tdt = average_daily_testing_proportion*A_u - omega_r*A_t - gamma_u*A_t


        dA_posdt = omega_r*A_t - gamma_t*A_pos




        return [dXdt, dEdt, dSdt, dA_udt, dA_tdt, dA_posdt]

    t_points = np.linspace(0, 30 * 365, 30 * 365) #30 years


    # ODE solver parameters
    abserr = 1.0e-8
    relerr = 1.0e-6


    solutions=np.zeros((num_trials,len(t_points), 6))
    sim_params=np.zeros((num_trials, len(t_points),8))

    for i in range(num_trials):
        solutions[i][0]= [X0, E0, S0, A_u0, A_t0, A_pos0]
        
    # Run Simulations

    for i in range(num_trials):
        for j in range(1, len(t_points)):



            #Stochastic params
            
            # Infection rate is 0% probability of infection per act(drawn from a beta dist) 
            beta = np.random.beta(estBetaParams(0.85,0.1)[0],estBetaParams(0.85,0.1)[1])
            
            #number of partners per day drawn from a gamma dist with mean =4 (per month) resclaled to 30 days. 
            # This seems a bit high but not unusual for particularly active groups of GBMSM people
            # Note, a higher variance may be appropriate here. 
            ppd = np.random.gamma(estGammaParams(3.05,3)[0],estGammaParams(3.05,3)[1])/30   
            
            # Inverse of the latency period post infection. Latency estimated to be gamma with mean 5 days
            epsilon = 1/(np.random.gamma(estGammaParams(5,1)[0],estGammaParams(5,1)[1])+0.000000000000001)  
            
            # Clearance rate with treatment. Infection is usually assumed to be resolved in an average of 
            # 7 days with standard treatment
            gamma_t = 1/(np.random.gamma(estGammaParams(7,1)[0],estGammaParams(7,1)[1])+0.00000000000001) 
            
            # Clearance rate for untreated individuals. We assume that most (asymptomatic) individuals will self-clear
            # in around two weeks, or else become symptomatic. Note the higher variance. 
            gamma_u = 1/(np.random.gamma(estGammaParams(14,3)[0],estGammaParams(14,3)[1])+0.00000000000001)
            
            theta = np.random.beta(estBetaParams(0.1,0.01)[0],estBetaParams(0.1,0.01)[1])
             
            lambda_ = np.random.beta(estBetaParams(0.9,0.01)[0],estBetaParams(0.9,0.01)[1]) 
            
            omega_r = 1/(np.random.gamma(estGammaParams(tau,1)[0], estGammaParams(tau,1)[1])+0.00000000000001)
            

            #record these for average yearly rates later.
            sim_params[i][j]=[beta,ppd,epsilon,gamma_t,theta,gamma_u,lambda_,omega_r]


            tspan = [t_points[j-1], t_points[j]]

            ys = odeint(model, solutions[i][j-1], tspan, args=(beta, ppd, epsilon, gamma_t, gamma_u, lambda_, theta, omega_r, average_daily_testing_proportion, tau, N), atol=abserr, rtol=relerr)

             # Update the solution
            solutions[i][j] = ys[-1]
            #print(i,j, solutions[i][j], sum(solutions[i][j]))


           # Initialize lists to store totals for each simulation
    total_infections_per_simulation = []
    total_detected_cases_per_simulation = []
    percent_detected_cases_per_simulation = []

    for i in range(num_trials):
        # Extract the last year's data for this simulation
        last_year_data = solutions[i][-365:]#/N ###MAKE SURE THIS DOESNT WRECK THINGS
        last_year_params = sim_params[i][-365:]

        # Initialize variables to store the sum of daily inflows
        sum_daily_inflow_E = 0
        sum_daily_inflow_S = 0
        sum_daily_inflow_A_pos = 0

        # Calculate daily inflow rates for the last year of this simulation
        for day_data, day_params in zip(last_year_data, last_year_params):
            X, E, S, A_u, A_t, A_pos = day_data
            beta,ppd, epsilon, gamma_t, theta, gamma_u, lambda_, omega_r = day_params

            # Compute daily inflow rates
            daily_inflow_E = beta* ppd * X * (A_u + A_t) / N
            daily_inflow_S = (1 - lambda_) * epsilon * E + theta * gamma_u * A_u
            daily_inflow_A_pos = omega_r * A_t

            # Sum the inflows
            sum_daily_inflow_E += daily_inflow_E
            sum_daily_inflow_S += daily_inflow_S
            sum_daily_inflow_A_pos += daily_inflow_A_pos

        # Calculate total yearly inflow for each compartment
        total_yearly_inflow_E = sum_daily_inflow_E
        total_yearly_inflow_S = sum_daily_inflow_S
        total_yearly_inflow_A_pos = sum_daily_inflow_A_pos

        # Total infections for the last year
        total_infections_last_year = total_yearly_inflow_E

        # Total Detected Cases 
        total_detected_cases = total_yearly_inflow_S + total_yearly_inflow_A_pos
        
        # Percent Detected Cases
        percent_detected_cases = total_detected_cases/total_infections_last_year

        # Store the totals for each simulation
        total_infections_per_simulation.append(total_infections_last_year)
        total_detected_cases_per_simulation.append(total_detected_cases)
        percent_detected_cases_per_simulation.append(percent_detected_cases)

    # Calculate the stats across all simulations
    
    #now compute averages accross all simulations:
    from scipy import stats
    average_total_infections = np.mean(total_infections_per_simulation)
    stdev_total_infections=np.std(total_infections_per_simulation)
    conf_total_infections=stats.norm.interval(0.95,loc=average_total_infections,scale=stdev_total_infections/np.sqrt(num_trials))
    upper_conf_bound_total_infections= average_total_infections+stats.norm.ppf(1-.05)*stdev_total_infections/np.sqrt(num_trials)
    total_infection_stats=[average_total_infections,stdev_total_infections, conf_total_infections, upper_conf_bound_total_infections]
    
    average_total_detected_cases = np.mean(total_detected_cases_per_simulation)
    stdev_total_detected_cases=np.std(total_detected_cases_per_simulation)
    conf_total_detected_cases=stats.norm.interval(0.95,loc=average_total_detected_cases,scale=stdev_total_detected_cases/np.sqrt(num_trials))
    upper_conf_bound_total_detected_cases = average_total_detected_cases+ stats.norm.ppf(1-.05)*stdev_total_detected_cases/np.sqrt(num_trials)
    total_detected_cases_stats=[average_total_detected_cases,stdev_total_detected_cases, conf_total_detected_cases,upper_conf_bound_total_detected_cases]
    
    average_percent_detected_cases = np.mean(percent_detected_cases_per_simulation)
    stdev_percent_detected_cases = np.std(percent_detected_cases_per_simulation)
    conf_percent_detected_cases = stats.norm.interval(0.95,loc=average_percent_detected_cases, scale=stdev_percent_detected_cases/np.sqrt(num_trials))
    upper_conf_bound_percent_detected_cases = average_percent_detected_cases + stats.norm.ppf(1-.05)*stdev_percent_detected_cases/np.sqrt(num_trials)
    percent_detected_cases_stats = [average_percent_detected_cases, stdev_percent_detected_cases, conf_percent_detected_cases, upper_conf_bound_percent_detected_cases]
   
    #Average Solution Scaled
    
    average_solution=np.mean(solutions, axis=0)/N



    return average_solution, total_infection_stats, total_detected_cases_stats, percent_detected_cases_stats